In [1]:
import pandas as pd
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
from pyspark.sql.functions import lag
from pyspark.sql.window import Window
from pyspark.sql.functions import lead
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from pyspark.sql.functions import collect_list
from pyspark.sql.functions import datediff
from pyspark.sql.functions import row_number
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf, col
from pyspark.sql.types import ArrayType, DoubleType, StringType
from pyspark.sql.functions import coalesce
from sklearn.feature_extraction.text import CountVectorizer

### Data Extraction

In [2]:
start_date = '2017-03-01'
end_date = '2017-09-30'
region_id = 1  ## United states
id_col = 'qbo_company_id'
status_features = ['qbo_company_id', 'qbo_country', 'qbo_language_name', 'qbo_signup_date',
                   'qbo_subscription_type_desc',
                   'qbo_accountants_attached_fourtyfive', 'qbo_current_product', 'payroll_signup_date',
                   'payroll_cancel_date',
                   'payments_signup_date', 'payments_cancel_date']
usage_features = ['qbo_company_id', 'ftujob_exp', 'ftujob_stok', 'ftujob_ret', 'ftujob_inv', 'ftujob_bill',
                  'ftujob_stax', 'ftujob_emp', 'ftujob_time']
qbo_usage_table = 'sbg_sandbox.ipdrec_featureset'
qbo_status_table = 'sbg_published.qbo_company_status'

In [3]:
def offline_feature_collection(qbo_usage_table, qbo_status_table, status_features, usage_features, id_col, start_date, end_date, region_id):
    qbo_status = spark.table(qbo_status_table).select(status_features) \
        .where("qbo_signup_date>='{}' and qbo_signup_date<='{}' and qbo_region_id={}"\
               .format(start_date, end_date,region_id))
    qbo_usage = spark.table(qbo_usage_table).select(usage_features)

    static_features = qbo_status.join(qbo_usage, [id_col], 'left_outer')

    return static_features

In [4]:
static_features = offline_feature_collection(qbo_usage_table, qbo_status_table, status_features, usage_features, id_col, start_date, end_date, region_id)

In [5]:
df = static_features

In [6]:
drop_list = ['payroll_signup_date','payroll_cancel_date','payments_signup_date','payments_cancel_date',
             'qbo_accountants_attached_fourtyfive']

num_cols = ['ftujob_exp','ftujob_stok','ftujob_ret','ftujob_inv','ftujob_bill',
        'ftujob_stax','ftujob_emp','ftujob_time']
encoding_cols = encoding_cols = [#'qbo_country'，
    'qbo_language_name','qbo_subscription_type_desc','qbo_current_product']
fill_na_dict = {}
for c in num_cols:
    fill_na_dict[c] = 0

In [7]:
class Vectorizer(object):
    def __init__(self,encoding_cols=None,stage='modeling',transformers=[],labels=[]):
        self.encoding_cols = encoding_cols
        self.stage = stage
        self.transformers = transformers
        self.labels = labels
        
    def transform(self,df): 
        
        to_array_fun = udf(self.vector_to_array, ArrayType(DoubleType()))
        
        if self.stage=='modeling':
            indexers = []
            encoders = []
            for c in encoding_cols:
                indexer = StringIndexer(inputCol=c, outputCol=c+'_index',handleInvalid='skip') 
                indexer_model = indexer.fit(df)
                df = indexer_model.transform(df)
                self.labels.append(indexer_model.labels)
                encoder = OneHotEncoder(inputCol=c+"_index", outputCol=c+'_vec',dropLast=False)
                df = encoder.transform(df)
                df = df.withColumn(c+'_array',to_array_fun(c+'_vec'))
                indexers.append(indexer_model)
                encoders.append(encoder)
            
            self.transformers.append(indexers)
            self.transformers.append(encoders)

        elif self.stage=='scoring':
            indexers = self.transformers[0]
            encoders = self.transformers[1]
            
            for i in range(len(encoding_cols)):
                indexer_model = indexers[i]
                encoder = encoders[i]
                df = indexer_model.transform(df)
                df = encoder.transform(df)
                c = encoding_cols[i]
                df = df.withColumn(c+'_array',to_array_fun(c+'_vec'))
        
        for c in encoding_cols:
            df = df.drop(c)
            idx_col = c+"_index"
            df = df.drop(idx_col)
            vec_col = c+"_vec"
            df = df.drop(vec_col)
            
        ## split sparse vector from onehotencoding into multiple columns
        array_cols =[c+'_array' for c in encoding_cols]
        columns = list(set(df.columns)-set(array_cols))
        
        for i, c in enumerate(encoding_cols):
            label = self.labels[i]
            array_col = c +'_array'
            columns += [col(array_col)[j] for j in range(len(label))]
            
        df = df.select(columns)
        
        return df
      
    def get_transformers(self):
        return self.transformers
    
    def vector_to_array(self,c):
        return c.toArray().tolist()
    
    def get_labels(self):
        return self.labels


In [8]:
#from pyspark.sql.functions import udf, col

In [9]:
def payroll_payments_attach_flag(signup_date, cancel_date):
    if signup_date != None and cancel_date == None:
        return 1
    else:
        return 0   

def boolean_to_int(x):
    if x == True:
        return 1
    else:
        return 0

payroll_payments_attach_udf = udf(payroll_payments_attach_flag,IntegerType())
fun_boolean_to_int = udf(boolean_to_int,IntegerType())

df = df.withColumn('payroll_attach_flag',payroll_payments_attach_udf(df.payroll_signup_date,df.payroll_cancel_date))\
 .withColumn('payments_attach_flag',payroll_payments_attach_udf(df.payments_signup_date,df.payments_cancel_date))\
 .withColumn('qbo_accountants_attached_45',fun_boolean_to_int('qbo_accountants_attached_fourtyfive'))

for c in drop_list:
    df = df.drop(c)
df = df.fillna(fill_na_dict)
vec = Vectorizer(encoding_cols,stage='modeling',transformers=[],labels=[])
data = vec.transform(df)
vec.transformers = vec.get_transformers()
vec.stage='scoring'
labels = vec.get_labels()

In [10]:
data.printSchema()

root
 |-- ftujob_exp: integer (nullable = false)
 |-- ftujob_stax: integer (nullable = false)
 |-- payroll_attach_flag: integer (nullable = true)
 |-- ftujob_ret: integer (nullable = false)
 |-- ftujob_inv: integer (nullable = false)
 |-- ftujob_stok: integer (nullable = false)
 |-- ftujob_bill: integer (nullable = false)
 |-- qbo_country: string (nullable = true)
 |-- ftujob_emp: integer (nullable = false)
 |-- qbo_accountants_attached_45: integer (nullable = true)
 |-- qbo_signup_date: date (nullable = true)
 |-- ftujob_time: integer (nullable = false)
 |-- qbo_company_id: long (nullable = true)
 |-- payments_attach_flag: integer (nullable = true)
 |-- qbo_language_name_array[0]: double (nullable = true)
 |-- qbo_language_name_array[1]: double (nullable = true)
 |-- qbo_language_name_array[2]: double (nullable = true)
 |-- qbo_language_name_array[3]: double (nullable = true)
 |-- qbo_language_name_array[4]: double (nullable = true)
 |-- qbo_language_name_array[5]: double (nullable = 

In [11]:
data.limit(10).toPandas()

,ftujob_exp,ftujob_stax,payroll_attach_flag,ftujob_ret,ftujob_inv,ftujob_stok,ftujob_bill,qbo_country,ftujob_emp,qbo_accountants_attached_45,qbo_signup_date,ftujob_time,qbo_company_id,payments_attach_flag,qbo_language_name_array[0],qbo_language_name_array[1],qbo_language_name_array[2],qbo_language_name_array[3],qbo_language_name_array[4],qbo_language_name_array[5],qbo_language_name_array[6],qbo_language_name_array[7],qbo_language_name_array[8],qbo_language_name_array[9],qbo_language_name_array[10],qbo_language_name_array[11],qbo_language_name_array[12],qbo_language_name_array[13],qbo_language_name_array[14],qbo_language_name_array[15],qbo_language_name_array[16],qbo_language_name_array[17],qbo_language_name_array[18],qbo_language_name_array[19],qbo_language_name_array[20],qbo_language_name_array[21],qbo_subscription_type_desc_array[0],qbo_subscription_type_desc_array[1],qbo_subscription_type_desc_array[2],qbo_subscription_type_desc_array[3],qbo_subscription_type_desc_array[4],qbo_current_product_array[0],qbo_current_product_array[1],qbo_current_product_array[2]
0,0,0,0,0,0,0,0,United States,0,0,2017-08-18,0,416068341,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0,0,0,0,0,0,0,United States,0,0,2017-04-21,0,416440391,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0,0,0,0,0,0,0,United States,0,0,2017-04-21,0,416440426,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0,0,0,0,0,0,0,United States,0,0,2017-04-11,0,417097776,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0,0,0,0,0,0,0,United States,0,0,2017-04-27,0,417098196,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0,0,0,0,0,0,0,United States,0,0,2017-05-17,0,417099021,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,0,0,0,0,0,0,0,United States,0,0,2017-04-11,0,1445344025,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,0,0,0,0,0,0,0,United States,0,0,2017-08-28,0,123145746916604,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,0,0,0,0,0,0,0,United States,0,0,2017-03-18,0,123145785962629,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
9,0,0,0,0,0,0,0,United States,0,0,2017-03-18,0,123145785965049,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
data.write.save('/user/xmeng-admin/qbo_pre_search/static_features_7m',format='parquet',mode='overwrite')

In [13]:
data = spark.read.format('parquet').load('/user/xmeng-admin/qbo_pre_search/static_features_7m')

In [14]:
data.count()

725707

In [15]:
start_date_clk = '2017-03-01'
end_date_clk = '2017-11-25'
clk_table = 'SBG_DWH_V2.TRANS_CLICKSTREAM'
reportsuite = 'HARMONY'
event_date = 'event_date'
static_feature_path = '/user/xmeng-admin/qbo_pre_search/static_features_7m'
id_col = 'qbo_company_id'
event_date = 'event_date'

In [17]:
def clk_feature_collection(static_feature_path,clk_table,start_date,end_date,reportsuite):
    static_features = spark.read.format('parquet').load(static_feature_path)
    static_features.createOrReplaceTempView("static_feature")
    clk_raw = spark.sql("\
    select evar59 as qbo_company_id \
    , trans_id \
    , hit_timestamp \
    , event_date \
    , evar27 as existing_transaction \
    , evar65 as raw_action \
    , evar64 \
    , post_evar64 \
    , evar12 \
    , evar13 \
    , evar25 \
    , case when evar65 like 'startipd_addpayroll%' THEN '1' \
        when evar65 like 'seeyourplan_chooseplan.product:%' THEN '2' \
        when evar65 like 'start_payday.flow:regular_trowser_paydayflowstep1view' THEN '3' \
        when evar65 like 'next_finishbuttonclicked_footer_paydayflowstep3view' THEN '4' \
        when evar65 like 'navigate_addemployeelink_addemployeelink_paydayflowstep1view' THEN '5' \
        when evar65 like 'done_qbopaddeditemployee_trowser_' THEN '6' \
        when evar65 like 'create_qbopaddeditemployee_gridheader_finishddpopup_emplist' THEN '7' \
        when evar65 like 'done_qbopaddeditemployee_trowser_addeesetup' THEN '8' \
        when evar65 like 'edit_qbopaddeditemployee_gridheader_empdetails' THEN '9' \
        when evar65 like 'start_payday.flow:bonus_trowser_paydayflowstep1view' THEN '10' \
        when evar65 like 'clickdelete_employee.paycheckcorrections:buttonaction_paychecklist_empdetails' THEN '11' \
        when evar65 like 'clickdeletepaycheck_employee.paycheckcorrections:deletemodal_modal_empdetails' THEN '12' \
        when evar65 like 'create_timeactivity%' THEN '13' \
        when evar65 like 'saveandclosebtnclick_singletimesheets%' THEN '14' \
        when evar65 like 'saveandnewbtnclick_singletimesheets%' THEN '15' \
        when evar65 like 'saveandstaybtnclick_singletimesheets%' THEN '16' \
        when evar65 like 'close_timeactivity%' THEN '17' \
        when evar65 like 'create_timetracking%' THEN '18' \
        when evar65 like 'saveandclosebtnclick_weeklytimesheet%' THEN '19' \
        when evar65 like 'saveandnewbtnclick_weeklytimesheet%' THEN '20' \
        when evar65 like 'saveandstaybtnclick_weeklytimesheet%' THEN '21' \
        when evar65 like 'close_timetracking%' THEN '22' \
        when evar65 like 'create_invoice%' THEN '23' \
        when evar65 like 'saveandclose_invoice%' THEN '24' \
        when evar65 like 'saveandnew_invoice%' THEN '25' \
        when evar65 like 'saveandstay_invoice%' THEN '26' \
        when evar65 like 'close_invoice%' THEN '27' \
        when evar65 like 'cancel_invoice%' THEN '28' \
        when evar65 like 'create_recvpayment%' THEN '29' \
        when evar65 like 'saveandclose_recvpayment%' THEN '30' \
        when evar65 like 'saveandnew_recvpayment%' THEN '31' \
        when evar65 like 'close_recvpayment%' THEN '32' \
        when evar65 like 'cancel_recvpayment%' THEN '33' \
        when evar65 like 'create_estimate%' THEN '34' \
        when evar65 like 'saveandclose_estimate%' THEN '35' \
        when evar65 like 'saveandnew_estimate%' THEN '36' \
        when evar65 like 'saveandstay_estimate%' THEN '37' \
        when evar65 like 'close_estimate%' THEN '38' \
        when evar65 like 'cancel_estimate%' THEN '39' \
        when evar65 like 'create_creditmemo%' THEN '40' \
        when evar65 like 'saveandclose_creditmemo%' THEN '41' \
        when evar65 like 'saveandnew_creditmemo%' THEN '42' \
        when evar65 like 'saveandsend_creditmemo%' THEN '43' \
        when evar65 like 'close_creditmemo%' THEN '44' \
        when evar65 like 'cancel_creditmemo%' THEN '45' \
        when evar65 like 'create_salesreceipt%' THEN '46' \
        when evar65 like 'saveandstay_salesreceipt%' THEN '47' \
        when evar65 like 'saveandclose_salesreceipt%' THEN '48' \
        when evar65 like 'saveandnew_salesreceipt%' THEN '49' \
        when evar65 like 'saveandsend_salesreceipt%' THEN '50' \
        when evar65 like 'close_salesreceipt%' THEN '51' \
        when evar65 like 'cancel_salesreceipt%' THEN '52' \
        when evar65 like 'create_refundreceipt%' THEN '53' \
        when evar65 like 'saveandclose_refundreceipt%' THEN '54' \
        when evar65 like 'saveandnew_refundreceipt%' THEN '55' \
        when evar65 like 'saveandsend_refundreceipt%' THEN '56' \
        when evar65 like 'close_refundreceipt%' THEN '57' \
        when evar65 like 'cancel_refundreceipt%' THEN '58' \
        when evar65 like 'create_nonpostingcredit%' THEN '59' \
        when evar65 like 'saveandclose_nonpostingcredit%' THEN '60' \
        when evar65 like 'saveandnew_nonpostingcredit%' THEN '61' \
        when evar65 like 'close_nonpostingcredit%' THEN '62' \
        when evar65 like 'cancel_nonpostingcredit%' THEN '63' \
        when evar65 like 'create_nonpostingcharge%' THEN '64' \
        when evar65 like 'saveandclose_nonpostingcharge%' THEN '65' \
        when evar65 like 'saveandnew_nonpostingcharge%' THEN '66' \
        when evar65 like 'close_nonpostingcharge%' THEN '67' \
        when evar65 like 'cancel_nonpostingcharge%' THEN '68' \
        when evar65 like 'create_expense%' THEN '69' \
        when evar65 like 'saveandstay_expense%' THEN '70' \
        when evar65 like 'saveandclose_expense%' THEN '71' \
        when evar65 like 'saveandnew_expense%' THEN '72' \
        when evar65 like 'close_expense%' THEN '73' \
        when evar65 like 'cancel_expense%' THEN '74' \
        when evar65 like 'create_bill_createnew%' THEN '75' \
        when evar65 like 'saveandstay_bill_sticky%' THEN '76' \
        when evar65 like 'saveandclose_bill_sticky%' THEN '77' \
        when evar65 like 'saveandnew_bill_sticky%' THEN '78' \
        when evar65 like 'close_bill_trowser%' THEN '79' \
        when evar65 like 'cancel_bill_sticky%' THEN '80' \
        when evar65 like 'create_check%' THEN '81' \
        when evar65 like 'saveandclose_check%' THEN '82' \
        when evar65 like 'saveandnew_check%' THEN '83' \
        when evar65 like 'close_check%' THEN '84' \
        when evar65 like 'cancel_check%' THEN '85' \
        when evar65 like 'create_purchaseorder%' THEN '86' \
        when evar65 like 'saveandstay_purchaseorder%' THEN '87' \
        when evar65 like 'saveandclose_purchaseorder%' THEN '88' \
        when evar65 like 'saveandnew_purchaseorder%' THEN '89' \
        when evar65 like 'saveandsend_purchaseorder%' THEN '90' \
        when evar65 like 'close_purchaseorder%' THEN '91' \
        when evar65 like 'cancel_purchaseorder%' THEN '92' \
        when evar65 like 'create_vendorcredit%' THEN '93' \
        when evar65 like 'saveandclose_vendorcredit%' THEN '94' \
        when evar65 like 'saveandnew_vendorcredit%' THEN '95' \
        when evar65 like 'close_vendorcredit%' THEN '96' \
        when evar65 like 'cancel_vendorcredit%' THEN '97' \
        when evar65 like 'create_creditcardcredit%' THEN '98' \
        when evar65 like 'saveandclose_creditcardcredit%' THEN '99' \
        when evar65 like 'saveandnew_creditcardcredit%' THEN '100' \
        when evar65 like 'close_creditcardcredit%' THEN '101' \
        when evar65 like 'cancel_creditcardcredit%' THEN '102' \
        when evar65 like 'create_deposit%' THEN '103' \
        when evar65 like 'saveandclose_deposit%' THEN '104' \
        when evar65 like 'saveandnew_deposit%' THEN '105' \
        when evar65 like 'close_deposit%' THEN '106' \
        when evar65 like 'cancel_deposit%' THEN '107' \
        when evar65 like 'start_manualupdate%' THEN '108' \
        when evar65 like 'complete_manualupdate%' THEN '109' \
        when evar65 like 'start_bankfileupload%' THEN '110' \
        when evar65 like 'complete_finishfileupload%' THEN '111' \
        when evar65 like 'close_bankfileupload%' THEN '112' \
        when evar65 like 'cancel_cancelfileupload%' THEN '113' \
        when evar65 like 'create_account%' THEN '114' \
        when evar65 like 'save_account%' THEN '115' \
        when evar65 like 'cancel_account%' THEN '116' \
        when evar65 like 'select_subscribe.acctstatus:%_accountingbilling_%' THEN '117' \
        when evar65 like 'success_subscribe.addproll:%' THEN '118' \
        when evar65 like 'navigate_upgradeaccount.acctstatus:%%%_accountingbilling_%' THEN '119' \
        when evar65 like 'navigate_upgradeaccount.acctstatus:%_%' THEN '120' \
        when evar65 like 'select_switchbillfreq.currfreq:%' THEN '121' \
        when evar65 like 'success_switchbillfreq%' THEN '122' \
        when evar65 like 'select_removepayroll_subscribe_%' THEN '123' \
        when evar65 like 'successpayroll_cancelworkflow_%' THEN '124' \
        when evar65 like 'edit_companyname.tab:company_settings%' THEN '125' \
        when evar65 like 'save_settings.sectionname:companyname.fields%' THEN '126' \
        when evar65 like 'edit_advancedtaxform.tab:company_settings%' THEN '127' \
        when evar65 like 'save_settings.sectionname:advancedtaxform.fields%' THEN '128' \
        when evar65 like 'edit_advancedchartofaccounts.tab:advanced_settingsubsection%' THEN '129' \
        when evar65 like 'save_settings.sectionname:advancedchartofaccounts.fields%' THEN '130' \
        when evar65 like 'edit_contactinfo.tab:company_settings%' THEN '131' \
        when evar65 like 'save_settings.sectionname:contactinfo.field%' THEN '132' \
        when evar65 like 'edit_companyaddress.tab:company_settings%' THEN '133' \
        when evar65 like 'save_settings.sectionname:companyaddress.field%' THEN '134' \
        when evar65 like 'edit_advancedaccounting.tab:advanced_settingsubsection%' THEN '135' \
        when evar65 like 'save_settings.sectionname:advancedaccounting.fields%' THEN '136' \
        when evar65 like 'edit_advancedtaxform.tab:advanced_settingsubsection%' THEN '137' \
        when evar65 like 'save_settings.sectionname:advancedtaxform.fields%' THEN '138' \
        when evar65 like 'edit_advancedtracking.tab:advanced_settingsubsection%' THEN '139' \
        when evar65 like 'save_settings.sectionname:advancedtracking.fields%' THEN '140' \
        when evar65 like 'edit_advancedaccounting.tab:advanced_settingsubsection%' THEN '141' \
        when evar65 like 'save_settings.sectionname:advancedautomation.fields%' THEN '142' \
        when evar65 like 'edit_advancedtimetracking.tab:advanced_settingsubsection%' THEN '143' \
        when evar65 like 'save_settings.sectionname:advancedtimetracking.fields%' THEN '144' \
        when evar65 like 'edit_currency.tab:advanced_settingsubsection%' THEN '145' \
        when evar65 like 'save_settings.sectionname:currency.fields%' THEN '146' \
        when evar65 like 'edit_advancedmisc.tab:advanced_settingsubsection%' THEN '147' \
        when evar65 like 'save_settings.sectionname:advancedmisc.fields%' THEN '148' \
        when evar65 like 'edit_salesform.tab:sales_%' THEN '149' \
        when evar65 like 'save_settings.sectionname:salesform.fields:%_settingsubsection%' THEN '150' \
        when evar65 like 'edit_salescommunications.tab:sales_%' THEN '151' \
        when evar65 like 'save_settings.sectionname:salescommunications.fields:%_settingsubsection%' THEN '152' \
        when evar65 like 'edit_remindercommunications.tab:sales_%' THEN '153' \
        when evar65 like 'save_settings.sectionname:remindercommunications.fields:%_settingsubsection%' THEN '154' \
        when evar65 like 'edit_salesonlinedelivery.tab:sales_%' THEN '155' \
        when evar65 like 'save_settings.sectionname:salesonlinedelivery.fields:%_settingsubsection%' THEN '156' \
        when evar65 like 'edit_salesstatement.tab:sales_%' THEN '157' \
        when evar65 like 'save_settings.sectionname:salesstatement.fields:%_settingsubsection%' THEN '158' \
        when evar65 like 'edit_bills.tab:expenses_%' THEN '159' \
        when evar65 like 'save_settings.sectionname:bills.fields:%_settingsubsection%' THEN '160' \
        when evar65 like 'edit_purchaseorders.tab:expenses_%' THEN '161' \
        when evar65 like 'save_settings.sectionname:purchaseorders.fields:%_settingsubsection%' THEN '162' \
        when evar65 like 'edit_expensecommunications.tab:expenses_%' THEN '163' \
        when evar65 like 'save_settings.sectionname:expensecommunications.fields:%_settingsubsection%' THEN '164' \
        when evar65 like 'create_transfer%' THEN '165' \
        when evar65 like 'saveandclose_transfer%' THEN '166' \
        when evar65 like 'saveandnew_transfer%' THEN '167' \
        when evar65 like 'close_transfer%' THEN '168' \
        when evar65 like 'cancel_transfer%' THEN '169' \
        when evar65 like 'create_journal%' THEN '170' \
        when evar65 like 'saveandstay_journal%' THEN '171' \
        when evar65 like 'saveandclose_journal%' THEN '172' \
        when evar65 like 'saveandnew_journal%' THEN '173' \
        when evar65 like 'close_journal%' THEN '174' \
        when evar65 like 'cancel_journal%' THEN '175' \
        when evar65 like 'create_statements%' THEN '176' \
        when evar65 like 'save_savestatements%' THEN '177' \
        when evar65 like 'saveandclose_saveandclosestatements%' THEN '178' \
        when evar65 like 'saveandsend_saveandsendstatements%' THEN '179' \
        when evar65 like 'close_statements%' THEN '180' \
        when evar65 like 'cancel_statements%' THEN '181' \
        when evar65 like 'create_homepage%' THEN '182' \
        when evar65 like 'saveandstay_inventory_quantity_adj%' THEN '183' \
        when evar65 like 'saveandclose_inventory_quantity_adj%' THEN '184' \
        when evar65 like 'saveandnew_inventory_quantity_adj%' THEN '185' \
        when evar65 like 'close_inventory_quantity_adj%' THEN '186' \
        when evar65 like 'cancel_inventory_quantity_adj%' THEN '187'  \
        when evar65 like 'canceltimesheet_singletimesheets%' THEN '188' \
        when evar65 like 'cancel_weeklytimesheet%' THEN '189' \
        when evar27 like '%qbo.intuit.com/app/bill?%txnid%' THEN '190' \
        when evar27 like '%qbo.intuit.com/app/check?%txnid%' THEN '191' \
        when evar27 like '%qbo.intuit.com/app/creditmemo?%txnid%' THEN '192' \
        when evar27 like '%qbo.intuit.com/app/deposit?%txnid%' THEN '193' \
        when evar27 like '%qbo.intuit.com/app/estimate?%txnid%' THEN '194' \
        when evar27 like '%qbo.intuit.com/app/expense?%txnid%' THEN '195' \
        when evar27 like '%qbo.intuit.com/app/invoice?%txnid%' THEN '196' \
        when evar27 like '%qbo.intuit.com/app/journal?%txnid%' THEN '197' \
        when evar27 like '%qbo.intuit.com/app/recvpayment?%txnid%' THEN '198' \
        when evar27 like '%qbo.intuit.com/app/refundreceipt?%txnid%' THEN '199' \
        when evar27 like '%qbo.intuit.com/app/salesreceipt?%txnid%' THEN '200' \
        when evar27 like '%qbo.intuit.com/app/timeactivity?%txnid%' THEN '201' \
        when evar27 like '%qbo.intuit.com/app/vendorcredit?%txnid%' THEN '202' \
        when evar27 like '%qbo.intuit.com/app/cashexpense?%txnid%' THEN '203' \
        when evar27 like '%qbo.intuit.com/app/creditcardcredit?%txnid%' THEN '204' \
        when evar27 like '%qbo.intuit.com/app/creditcardexpense?%txnid%' THEN '205' \
        when evar27 like '%qbo.intuit.com/app/inventory_quantity_adj?%txnid%' THEN '206' \
        when evar27 like '%qbo.intuit.com/app/nonpostingcharge?%txnid%' THEN '207' \
        when evar27 like '%qbo.intuit.com/app/nonpostingcredit?%txnid%' THEN '208' \
        when evar27 like '%qbo.intuit.com/app/purchaseorder?%txnid%' THEN '209' \
        when evar27 like '%qbo.intuit.com/app/transfer?%txnid%' THEN '210' \
        when evar27 like '%qbo.intuit.com/app/timetracking?%txnid%' THEN '211' \
        when evar65 like 'create_paybills%' THEN '212' \
        when evar27 like '%qbo.intuit.com/app/billpayment?%txnid%' THEN '213' \
        when evar65 like 'saveandclose_billpayment%' THEN '214' \
        when evar65 like 'saveandnew_billpayment%' THEN '215' \
        when evar65 like 'saveandstay_billpayment%' THEN '216' \
        when evar65 like 'close_billpayment%' THEN '217' \
        when evar65 like 'cancel_billpayment%' THEN '218' \
        when evar65 like 'saveandprint_billpayment%' THEN '219' \
        when evar65 like 'delete_bill_sticky%' THEN '220' \
        when evar65 like 'delete_check%' THEN '221' \
        when evar65 like 'delete_creditmemo%' THEN '222' \
        when evar65 like 'delete_deposit%' THEN '223' \
        when evar65 like 'delete_estimate%' THEN '224' \
        when evar65 like 'delete_expense%' THEN '225' \
        when evar65 like 'delete_invoice%' THEN '226' \
        when evar65 like 'delete_journal%' THEN '227' \
        when evar65 like 'delete_recvpayment%' THEN '228' \
        when evar65 like 'delete_refundreceipt%' THEN '229' \
        when evar65 like 'delete_salesreceipt%' THEN '230' \
        when evar65 like 'delete_vendorcredit%' THEN '231' \
        when evar65 like 'delete_cashexpense%' THEN '232' \
        when evar65 like 'delete_creditcardcredit%' THEN '233' \
        when evar65 like 'delete_creditcardexpense%' THEN '234' \
        when evar65 like 'delete_inventory_quantity_adj%' THEN '235' \
        when evar65 like 'delete_nonpostingcharge%' THEN '236' \
        when evar65 like 'delete_nonpostingcredit%' THEN '237' \
        when evar65 like 'delete_purchaseorder%' THEN '238' \
        when evar65 like 'delete_transfer%' THEN '239' \
        when evar65 like 'void_bill_sticky%' THEN '240' \
        when evar65 like 'void_check%' THEN '241' \
        when evar65 like 'void_creditmemo%' THEN '242' \
        when evar65 like 'void_deposit%' THEN '243' \
        when evar65 like 'void_estimate%' THEN '244' \
        when evar65 like 'void_expense%' THEN '245' \
        when evar65 like 'void_invoice%' THEN '246' \
        when evar65 like 'void_journal%' THEN '247' \
        when evar65 like 'void_recvpayment%' THEN '248' \
        when evar65 like 'void_refundreceipt%' THEN '249' \
        when evar65 like 'void_salesreceipt%' THEN '250' \
        when evar65 like 'void_vendorcredit%' THEN '251' \
        when evar65 like 'void_cashexpense%' THEN '252' \
        when evar65 like 'void_creditcardcredit%' THEN '253' \
        when evar65 like 'void_creditcardexpense%' THEN '254' \
        when evar65 like 'void_inventory_quantity_adj%' THEN '255' \
        when evar65 like 'void_nonpostingcharge%' THEN '256' \
        when evar65 like 'void_nonpostingcredit%' THEN '257' \
        when evar65 like 'void_purchaseorder%' THEN '258' \
        when evar65 like 'void_transfer%' THEN '259' \
        when evar65 like 'delete_billpayment%' THEN '260' \
        when evar65 like 'saveandprint_createbillpayments%' THEN '261' \
        when evar65 like 'close_paybills%' THEN '262' \
        when evar65 like 'create_bill_glob%' THEN '263' \
        else null end as task_id \
    from " + clk_table + " a \
    inner join static_feature b on a.evar59 = b.qbo_company_id \
    where a.event_date>='" + start_date + "' \
    and a.event_date<='" + end_date + "' \
    and a.reportsuite_name='" + reportsuite + "' \
    and a.evar59 is not null and a.evar59 <> ''")
    
    return clk_raw

In [18]:
clk_raw = clk_feature_collection(static_feature_path,clk_table,start_date_clk,end_date_clk,reportsuite)
clk_raw = clk_raw.repartition(id_col,event_date)

In [ ]:
clk_raw.write.save('/user/xmeng-admin/qbo_pre_search/clk_raw_7m',format='parquet',mode='overwrite')

In [ ]:
clk_raw = spark.read.format('parquet').load('/user/xmeng-admin/qbo_pre_search/clk_raw_7m')

In [ ]:
clk_raw.cache()

In [ ]:
count = clk_raw.count()

In [24]:
count

2522217170

In [25]:
CONTEXTUAL_FEATURE = 'task_id'
TARGET_FEATURE = 'article_id'
ADJ_TARGET_FEATURE = 'article_id_adj'
trans_id = 'trans_id'
timestamp = 'hit_timestamp'
k = -10
mapping_path = 'care-PON/article_mapping'

In [26]:
def article_title(s):
    if s !=None and s!='':
        s_list = s.split('topic:',1)
        if len(s_list)>1:
            topic_str = s_list[1]
            topic = topic_str.split('.')[0]
            return topic
        else:
            return ''
    else:    
        return s
article_title_fun = udf(article_title,StringType())
def article_id(s):
    if s !=None and s!='':
        s_list = s.split('topicid:',1)
        if len(s_list)>1:
            id_str = s_list[1]
            topic_id = id_str.split('.')[0]
            return topic_id
        else:
            return ''
    else:
        return s
article_id_fun = udf(article_id,StringType())

In [27]:
## input format from streaming should be company_id,event_date,trans_id,task_list

### Data cleaning

In [28]:
def clk_transform(clk_raw,id_col,event_date,timestamp,trans_id,CONTEXTUAL_FEATURE,TARGET_FEATURE,ADJ_TARGET_FEATURE,k,mapping_path):
    
    window = Window.partitionBy(id_col,event_date).orderBy(timestamp,trans_id)
    last_k_window = Window.partitionBy(id_col,event_date).orderBy(timestamp,trans_id).rowsBetween(k,0)
    row_num_window = Window.partitionBy(id_col,event_date,ADJ_TARGET_FEATURE).orderBy(timestamp,trans_id)
    
    ## lag task_id to compare previous task_id and current task_id, only keep record when there is a task change or when
    ## when the article_id is not null
    df = clk_raw.withColumn("last_{}".format(CONTEXTUAL_FEATURE),lag(CONTEXTUAL_FEATURE).over(window))\
            .withColumn('evar25_title',article_title_fun('evar25'))\
            .withColumn('evar25_id',article_id_fun('evar25'))\
            .withColumn('article_title',coalesce('evar12','evar25_title'))\
            .withColumn('article_id',coalesce('evar13','evar25_id'))\
            .selectExpr("*", "(case when {0} is not null or last_{1} is null or {1} <> last_{1} then 1 else 0 end) as flag_new_{1}"\
                        .format(TARGET_FEATURE,CONTEXTUAL_FEATURE))\
            .where("flag_new_{}=1".format(CONTEXTUAL_FEATURE))\
            .withColumn("{}_list".format(CONTEXTUAL_FEATURE),collect_list(CONTEXTUAL_FEATURE).over(last_k_window))\
            .where("article_id is not null and article_id <> ''")    
    
    article= spark.read.format('parquet').load(mapping_path)\
             .where((col('PROD_QBO')!='null')&((col('REGION_US')!='null')|(col('REGION_NONE')!='null'))&(col('state')=='live'))\
             .select(['article_id','article_id_adj'])
    data = df.join(article,['article_id'],'inner')
        
    data_filtered =data.withColumn("row_num",row_number().over(row_num_window))\
            .where("row_num=1 and article_id_adj is not null") 
    drop_cols = ['evar12','evar13','existing_transaction','raw_action','evar64','post_evar64',
                 'evar25','evar25_title','evar25_id','task_id','last_task_id','flag_new_task_id','row_num'
                ,'article_id','article_title','trans_id','hit_timestamp']    
    
    for c in drop_cols:
        data_filtered = data_filtered.drop(c) 
            
    return data_filtered

In [29]:
clk_filtered = clk_transform(clk_raw,id_col,event_date,timestamp,trans_id,CONTEXTUAL_FEATURE,TARGET_FEATURE,ADJ_TARGET_FEATURE,k,mapping_path)

In [30]:
clk_filtered.printSchema()

root
 |-- qbo_company_id: string (nullable = true)
 |-- event_date: date (nullable = true)
 |-- task_id_list: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- article_id_adj: integer (nullable = true)



In [31]:
clk_filtered.write.save('/user/xmeng-admin/qbo_pre_search/clk_filtered_7m',format='parquet',mode='overwrite')

In [32]:
clk_filtered = spark.read.format('parquet').load('/user/xmeng-admin/qbo_pre_search/clk_filtered_7m')

In [33]:
clk_filtered.count()

974994

### Feature combination

In [2]:
clk_path = '/user/xmeng-admin/qbo_pre_search/clk_filtered_7m'
static_path = '/user/xmeng-admin/qbo_pre_search/static_features_7m'

def combine_features(clk_path,static_path):
    clk = spark.read.format('parquet').load(clk_path)
    static_features = spark.read.format('parquet').load(static_path)
    
    df = clk.join(static_features,['qbo_company_id'],'inner')
    
    df = df.withColumn('tenure',datediff(df.event_date,df.qbo_signup_date))
    
    cols = ['qbo_signup_date','event_date','qbo_country']
    
    for c in cols:
        df = df.drop(c)
        
    return df

In [3]:
data = combine_features(clk_path,static_path)

In [4]:
data.count()

975268

### Transformer

In [5]:
df =  data.toPandas()

In [6]:
df1 = df.copy()

In [7]:
df1.head(2)

,qbo_company_id,task_id_list,article_id_adj,ftujob_exp,ftujob_stax,payroll_attach_flag,ftujob_ret,ftujob_inv,ftujob_stok,ftujob_bill,ftujob_emp,qbo_accountants_attached_45,ftujob_time,payments_attach_flag,qbo_language_name_array[0],qbo_language_name_array[1],qbo_language_name_array[2],qbo_language_name_array[3],qbo_language_name_array[4],qbo_language_name_array[5],qbo_language_name_array[6],qbo_language_name_array[7],qbo_language_name_array[8],qbo_language_name_array[9],qbo_language_name_array[10],qbo_language_name_array[11],qbo_language_name_array[12],qbo_language_name_array[13],qbo_language_name_array[14],qbo_language_name_array[15],qbo_language_name_array[16],qbo_language_name_array[17],qbo_language_name_array[18],qbo_language_name_array[19],qbo_language_name_array[20],qbo_language_name_array[21],qbo_subscription_type_desc_array[0],qbo_subscription_type_desc_array[1],qbo_subscription_type_desc_array[2],qbo_subscription_type_desc_array[3],qbo_subscription_type_desc_array[4],qbo_current_product_array[0],qbo_current_product_array[1],qbo_current_product_array[2],tenure
0,123145785961564,"[38, 194]",1429715,0,0,0,0,0,0,0,0,1,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2
1,123145785962809,[],1464993,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,47


In [8]:
class Vectorizer(object):
    def __init__(self,feature_list=None,stage='modeling'):
        self.feature_list = feature_list
        self.stage = stage
        
    def transform(self,df):                                
        
        df['task_sequence'] = df.apply(lambda row: self.task_sequence(row['task_id_list']),axis=1) 
        
        cv = CountVectorizer(stop_words=None, ngram_range=(1,1), analyzer='word')
        dt_mat = cv.fit_transform(df.task_sequence)
        df_mat = pd.DataFrame(dt_mat.todense(), index=df.index, columns=cv.get_feature_names())
        
        data = df.join(df_mat)
        
        cols = ['task_id_list','task_sequence']
        for c in cols:
            del data[c]
        
        if self.feature_list is not None:
            set1 = list(set(data.columns) - set(self.feature_list))
            if len(set1) > 0:
                #logger.debug( '----diff:unseen levels in training set: %s' % (','.join(set1)))
                for col in set1:
                    del data[col]
                    
            set2 = list(set(self.feature_list) - set(data.columns))
            if len(set2) > 0:
                #logger.debug( '----diff:fill column with all 0s: %s' % (','.join(set2)))
                for col in set2:
                    data[col] = 0
                    
        data.set_index('qbo_company_id',inplace=True)
                    
        return data
    
    def task_sequence(self,task_list):
        result = []
        for task in task_list:
            if task is not None:
                result.append(task)
        return ' '.join(result)

In [9]:
vec = Vectorizer()

In [10]:
df_all = vec.transform(df1)

In [11]:
df_all.shape

(975268, 278)

In [12]:
df_all.head(2)

,article_id_adj,ftujob_exp,ftujob_stax,payroll_attach_flag,ftujob_ret,ftujob_inv,ftujob_stok,ftujob_bill,ftujob_emp,qbo_accountants_attached_45,ftujob_time,payments_attach_flag,qbo_language_name_array[0],qbo_language_name_array[1],qbo_language_name_array[2],qbo_language_name_array[3],qbo_language_name_array[4],qbo_language_name_array[5],qbo_language_name_array[6],qbo_language_name_array[7],qbo_language_name_array[8],qbo_language_name_array[9],qbo_language_name_array[10],qbo_language_name_array[11],qbo_language_name_array[12],qbo_language_name_array[13],qbo_language_name_array[14],qbo_language_name_array[15],qbo_language_name_array[16],qbo_language_name_array[17],qbo_language_name_array[18],qbo_language_name_array[19],qbo_language_name_array[20],qbo_language_name_array[21],qbo_subscription_type_desc_array[0],qbo_subscription_type_desc_array[1],qbo_subscription_type_desc_array[2],qbo_subscription_type_desc_array[3],qbo_subscription_type_desc_array[4],qbo_current_product_array[0],qbo_current_product_array[1],qbo_current_product_array[2],tenure,10,100,101,102,103,104,105,106,107,108,109,11,110,111,112,113,114,115,116,117,118,119,12,121,122,123,124,125,126,127,128,129,13,130,131,132,133,134,135,136,137,139,14,140,142,143,144,145,147,148,149,15,150,151,152,153,154,155,156,157,158,159,16,160,161,162,163,164,165,166,167,168,169,17,170,171,172,173,174,175,176,177,178,179,18,180,182,183,184,185,186,187,188,189,19,190,191,192,193,194,195,196,197,198,199,20,200,201,202,204,205,206,207,208,209,21,210,212,213,214,215,217,218,219,22,220,221,222,223,224,225,226,227,228,229,23,230,231,233,234,235,236,237,238,239,24,241,242,245,246,248,249,25,250,253,254,259,26,260,261,262,263,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
qbo_company_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
123145785961564,1429715,0,0,0,0,0,0,0,0,1,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
123145785962809,1464993,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,47,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
df_all['ftujob_stax'].sum()

227633